# 1. 잠재 디리클레 할당(Latent Dirichlet Allcation, LDA) 개요
토픽 모델링의 대표적 알고리즘, LDA의 문서들은 토픽들의 혼합으로 구성되며, 토픽들은 확률분포에 기반하여 단어들을 생성한다고 가정   
데이터가 주어지면 LDA는 문서가 생성되던 과정을 역추적함   
LDA는 각 문서 토픽 분포와, 각 토픽 내 단어 분포를 추정   

문서1: 저는 사과랑 바나나를 먹어요   
문서2: 우리는 귀여운 강아지가 좋아요   
문서3: 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요   

<각 문서 토픽 분포>   
문서1: 토픽 A 100%   
문서2: 토픽 B 100%   
문서3: 토픽 B 60%, 토픽 A 40%   

<각 토픽 단어 분포>   
토픽A: 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜직하고 0%, 좋아요 0%   
토픽B: 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%   

LDA는 토픽 제목을 정해주진 않지만, 이 시점에서 알고리즘 사용자는 결과로부터 두 토픽을 판단 가능

# 2. LDA의 가정 
LDA는 문서 집합으로부터 어떤 토픽이 존재하는지 알아내기 위한 알고리즘   
BoW의 행렬 DTM 또는 TF-IDF 행렬을 입력으로 함, 단어의 순서는 신경쓰지 않음   
**'나는 문서를 작성하고자 이런 주제를 넣을거고, 이런 주제를 위해선 이런 단어를 넣을거야.**

> **1) 문서에 사용할 단어 개수 N을 정합니다.**   
> 예) 5개 단어를 정함

> **2) 문서에 사용할 토픽 혼합을 확률분포에 기반하여 결정합니다.**   
> 예) 위 예제와 같이 토픽이 2개일 때 강아지 토픽을 60%, 과일 토픽을 40%와 같이 선택

> **3) 문서에 사용할 각 단어를 정합니다.**
>> **3-1) 토픽 분포에서 토픽 T를 확률적으로 고릅니다.**   
>> 예) 60% 확률로 강아지 토픽 선택, 40% 확률로 과일 토픽 선택

>> **3-2) 선택한 토픽 T에서 단어의 출현 확률분포에 기반해 문서에 사용할 단어를 고릅니다.**   
>> 예) 강아지 토픽을 선택한다면, 33% 확률로 강아지란 단어 선택 가능

3)을 반복하며 문서를 완성   
이런 과정을 통해 문서가 작성된다는 가정 하에 LDA는 토픽을 뽑아내기 위해 위 과정을 역으로 추적하는 역공학(reverse engneering)을 수행

# 3. LDA의 수행하기
> **1) 사용자는 알고리즘에게 토픽 개수 k를 알려줌**   
> LDA는 토픽 개수 k를 입력받으면, k개 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정

> **2) 모든 단어를 k개 중 하나의 토픽에 할당**   
> 이 작업이 끝나면 각 문서는 토픽을 가지며, 토픽은 단어 분포를 갖게 됨   
> 물론 랜덤으로 할당하였기에 이 결과는 전부 틀린 상태   
> 만약 한 단어가 한 문서에서 2회 이상 등장했다면, 각 단어는 서로 다른 토픽에 할당되었을 수 있음

> **3) 모든 문서의 모든 단어에 대해 아래 사항을 iteration**   
>> **3-1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되었지만, 다른 단어들은 모두 올바른 토픽에 할당된 상태라고 가정함   
>> 이에 따라 단어 w는 아래 기준에 따라 토픽이 재할당됨   
>> 1. p(topic t|document d): 문서 d의 단어 중 토픽 t에 해당하는 단어들의 비율   
>> 2. p(word w|topic t): 각 토픽 t에서 해당 단어 w의 분포   

>> doc1 word|apple|banana|apple|dog|dog
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|B|B|???|A|A

>> doc2 word|cute|book|king|apple|apple
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|B|B|B|B|B

>> 여기서 doc1의 세번째 단어 apple의 토픽을 결정하는 상황   

>> doc1 word|**apple**|**banana**|apple|**dog**|**dog**
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|**B**|**B**|???|**A**|**A**

>> 첫째 기준은 doc1의 단어들이 어떤 토픽에 해당하는지 확인   
>> doc1의 모든 단어들은 토픽 A와 토픽 B에 50 대 50의 비율로 할당되어 있으므로, 기준에 따르면 단어 apple은 어떤 토픽에도 속할 가능성이 있음   

>> doc1 word|**apple**|banana|**apple**|dog|dog
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|**B**|B|**???**|A|A

>> doc2 word|cute|book|king|**apple**|**apple**
>> :-:|:-:|:-:|:-:|:-:|:-:
>> topic|B|B|B|**B**|**B**

>> 둘째 기준은 단어 apple이 전체 문서에서 어떤 토픽에 할당되었는지 확인   
>> 이 기준에 따르면 단어 apple은 토픽 B에 할당될 가능성이 높음

# 4. 잠재 의미 분석과 잠재 디리클레 할당의 차이
LSA: DTM의 축소되 차원에서 근접 단어들을 토픽으로 묶음   
LDA: 단어가 특정 토픽에 존재할 확률과, 문서에 특정 토픽이 존재할 확률을 결합확률로 추정해 토픽을 추출

# 5. 실습을 통한 이해
> **1) 정수 인코딩과 단어 집합 만들기**   

In [1]:
# 뉴스그룹 데이터 로드
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

# 텍스트 전처리
news_df = pd.DataFrame({'document':documents})

news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

<ipython-input-1-aedc00f00276>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [2]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [3]:
# 정수 인코딩 및 단어 빈도수 기록
from gensim import corpora 
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [4]:
print(dictionary[55])

accept


In [5]:
len(dictionary)

64281

> **2) LDA 모델 훈련시키기**   
> 단어 앞의 수치는 해당 토픽에 대한 단어의 기여도   
> passes는 알고리즘 동작 횟수, num_words로 출력할 단어 개수 조정

In [6]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.020*"windows" + 0.017*"problem" + 0.012*"using" + 0.011*"card"')
(1, '0.009*"smith" + 0.009*"ball" + 0.008*"mike" + 0.007*"steve"')
(2, '0.011*"information" + 0.007*"system" + 0.007*"data" + 0.007*"encryption"')
(3, '0.039*"armenian" + 0.032*"turkish" + 0.029*"armenians" + 0.020*"turkey"')
(4, '0.011*"myers" + 0.008*"andrew" + 0.007*"david" + 0.007*"navy"')
(5, '0.009*"president" + 0.007*"year" + 0.007*"health" + 0.006*"university"')
(6, '0.015*"drive" + 0.010*"price" + 0.009*"disk" + 0.009*"used"')
(7, '0.021*"would" + 0.013*"like" + 0.012*"people" + 0.011*"think"')
(8, '0.019*"period" + 0.017*"play" + 0.012*"power" + 0.012*"chicago"')
(9, '0.019*"please" + 0.018*"thanks" + 0.016*"mail" + 0.015*"software"')
(10, '0.025*"year" + 0.025*"game" + 0.023*"team" + 0.019*"games"')
(11, '0.032*"file" + 0.017*"program" + 0.016*"output" + 0.015*"window"')
(12, '0.029*"greek" + 0.014*"gordon" + 0.013*"pitt" + 0.013*"banks"')
(13, '0.010*"pain" + 0.010*"water" + 0.009*"ground" + 0.007*"wire

> **3) LDA 시각화하기**   
> LDA 모델 출력 결과에서는 토픽 번호가 0부터 할당되지만, LDA 시각화에서는 1부터 시작

In [7]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

> **4) 문서 별 토픽 분포 보기**

In [8]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i == 5:
        break
    print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(7, 0.2402769), (14, 0.7451981)]
1 번째 문서의 topic 비율은 [(1, 0.086492), (2, 0.031361435), (3, 0.054698404), (7, 0.546867), (12, 0.027552404), (18, 0.23458457)]
2 번째 문서의 topic 비율은 [(1, 0.018130874), (7, 0.54814494), (12, 0.077975325), (14, 0.3428227)]
3 번째 문서의 topic 비율은 [(2, 0.265762), (5, 0.09105168), (6, 0.13159081), (7, 0.37998998), (13, 0.07327721), (14, 0.04733863)]
4 번째 문서의 topic 비율은 [(1, 0.30160278), (7, 0.3693215), (8, 0.114430495), (10, 0.11415185), (11, 0.0727077)]


C:\Users\USER\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# 데이터프레임 형식으로 출력
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        
        for j, (topic_num, prop_topic) in enumerate(doc):
            if j == 0:
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), 
                                                            topic_list]), ignore_index=True)
            else:
                break
                
    return(topic_table)

C:\Users\USER\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index()
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

C:\Users\USER\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,14.0,0.7452,"[(7, 0.24027719), (14, 0.7451978)]"
1,1,7.0,0.5468,"[(1, 0.086494274), (2, 0.031353768), (3, 0.054..."
2,2,7.0,0.5482,"[(1, 0.018131495), (7, 0.5481681), (12, 0.0779..."
3,3,7.0,0.3800,"[(2, 0.2657631), (5, 0.09105103), (6, 0.131592..."
4,4,7.0,0.3693,"[(1, 0.30160037), (7, 0.36930245), (8, 0.11441..."
5,5,7.0,0.4134,"[(0, 0.07927837), (4, 0.18428011), (6, 0.06224..."
6,6,7.0,0.3805,"[(2, 0.016981723), (6, 0.30849835), (7, 0.3805..."
7,7,7.0,0.5147,"[(7, 0.5146909), (10, 0.12071181), (14, 0.1447..."
8,8,7.0,0.4361,"[(0, 0.042998888), (1, 0.055239882), (2, 0.066..."
9,9,7.0,0.4834,"[(6, 0.17802091), (7, 0.48341095), (13, 0.1866..."
